In [342]:
# Tian 1:10, 17:26
subcortical_index = list(range(0,10)) + list(range(16,26))
# Schaefer: 
# lh-mPFC: 199:205
# rh-mPFC: 464:470
# lh-Ins: 67, 108:111, 126:128
# rh-Ins: 319, 361:364, 383:386
## ACC: 390
# Glasser
cortical_roi = ['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins']
lh_dlPFC_index = [205, 246, 247, 249, 250, 252, 262, 263, 264, 265, 266, 276, 277]
rh_dlPFC_index = [25, 66, 67, 69, 70, 72, 82, 83, 84, 85, 86, 96, 97]
lh_mPFC_index = [236, 237, 238, 239, 240, 241, 242, 243, 244, 248, 267, 343, 344, 345, 358, 359]
rh_mPFC_index = [56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 87, 163, 164, 165, 178, 179]
lh_PCC_index = [193, 194, 206, 209, 210, 211, 212, 213, 214, 300, 321, 340, 341]
rh_PCC_index = [13, 14, 26, 29, 30, 31, 32, 33, 34, 120, 141, 160, 161]
lh_Ins_index = [285, 287, 288, 289, 290, 291, 293, 294, 346, 347, 348, 357]
rh_Ins_index = [105, 107, 108, 109, 110, 111, 113, 114, 166, 167, 168, 177]
dic_cortical_roi = {
    'lh_dlPFC': lh_dlPFC_index,
    'rh_dlPFC': rh_dlPFC_index,
    'lh_mPFC': lh_mPFC_index,
    'rh_mPFC': rh_mPFC_index,
    'lh_PCC': lh_PCC_index,
    'rh_PCC': rh_PCC_index,
    'lh_Ins': lh_Ins_index,
    'rh_Ins': rh_Ins_index
}

In [406]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure


# === Step 1: Define Functions === #

def load_dataset(base_dir, data_set):
    """
    Load dataset-specific files.
    """
    fMRIinfo_file_path = os.path.join(base_dir, f"{data_set}_data_set.csv")
    participant_file_path = os.path.join(base_dir, "participants_fMRI.csv")
    return pd.read_csv(fMRIinfo_file_path), pd.read_csv(participant_file_path)


def load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Load cortical and subcortical timeseries data for a subject.
    """
    cortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, "timeseries", cortical_file_name)

    subcortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, "timeseries", subcortical_file_name)

    if not (os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path)):
        print(f"Missing files for subject {subject_ID}, session {session_ID}.")
        return None

    # Load and process cortical timeseries
    df_cortical_all = pd.read_csv(cortical_file_path, compression="gzip", index_col=0, header=0)
    df_cortical_roi = pd.DataFrame({
        roi: df_cortical_all.iloc[dic_cortical_roi[roi]].mean(axis=0)
        for roi in dic_cortical_roi.keys()
    })

    # Load and process subcortical timeseries
    df_subcortical_all = pd.read_csv(subcortical_file_path, compression="gzip", index_col=0, header=0)
    df_subcortical_roi = df_subcortical_all.iloc[subcortical_index]

    # Combine cortical and subcortical ROIs
    return pd.concat([df_cortical_roi, df_subcortical_roi.transpose()], axis=1)


def clean_data(data):
    """
    Handle missing values and remove constant features from time series data.
    """
    # Fill NaNs with column-wise means
    data_filled = np.copy(data)
    for j in range(data.shape[1]):
        if np.isnan(data[:, j]).any():
            data_filled[:, j] = np.nan_to_num(data[:, j], nan=np.nanmean(data[:, j]))

    # Remove constant features
    non_constant_features = data_filled[:, data_filled.std(axis=0) != 0]
    return non_constant_features


def compute_connectivity(data):
    """
    Compute connectivity matrix for the given time series data.
    """
    correlation_measure = ConnectivityMeasure(kind="correlation")
    return correlation_measure.fit_transform([data])[0]


def extract_upper_triangle(matrix):
    """
    Extract the upper triangle values (excluding diagonal) from a connectivity matrix.
    """
    upper_tri_indices = np.triu_indices(matrix.shape[0], k=1)
    return matrix[upper_tri_indices]


# === Step 2: Define the Pipeline === #

def process_fMRI_subject(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing a single subject's fMRI data.
    """
    # Load subject timeseries
    df_roi = load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index)
    if df_roi is None:
        return None, None

    # Clean data
    #cleaned_data = clean_data(df_roi.values)

    # Standardize data
    standardized_data = StandardScaler().fit_transform(df_roi.values)

    # Compute connectivity matrix
    connectivity_matrix = compute_connectivity(standardized_data)

    # Extract upper triangle
    upper_triangle = extract_upper_triangle(connectivity_matrix)

    return upper_triangle, subject_ID


def process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing fMRI data for all subjects.
    """
    # Set paths
    base_dir = os.path.join(user_dir, project_name, "data")
    derivatives_dir = os.path.join(base_dir, "derivatives")

    # Load dataset
    df_fMRIinfo, df_participants = load_dataset(base_dir, data_set)
    subject_IDs = df_fMRIinfo["eid"].unique()

    # Initialize lists for data
    connectivity_data = []
    subject_ids_cleaned = []

    # Process each subject individually
    for subject_ID in subject_IDs:
        upper_triangle, cleaned_subject_ID = process_fMRI_subject(
            subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index
        )
        if upper_triangle is not None:
            connectivity_data.append(upper_triangle)
            subject_ids_cleaned.append(cleaned_subject_ID)

    # Filter participants based on available data
    df_filtered = df_participants.loc[df_participants["eid"].isin(subject_ids_cleaned)]

    return np.array(connectivity_data), df_filtered


In [434]:
#codes for modeling
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Ensure binary target
def ensure_binary_target(y):
    unique_values = np.unique(y)
    if len(unique_values) > 2:
        raise ValueError("Target variable contains more than two classes. Please preprocess the data.")
    if unique_values.dtype == bool:
        return y.astype(int)
    elif set(unique_values) == {0, 1} or set(unique_values) == {1, 0}:
        return y
    else:
        raise ValueError("Target variable is not binary. Please preprocess the data.")


# Split data into training and testing sets
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


# Model selection using cross-validation
def model_selection(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),
        "Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),
        "Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),
        "ElasticNet (L1+L2)": LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5),
        "LDA": LinearDiscriminantAnalysis(),
        "Perceptron": Perceptron(),
        "SVM (Linear)": SVC(kernel='linear'),
        "Random Forest": RandomForestClassifier(),
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    for model_name, model in models.items():
        cv_score = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy').mean()
        print(f"Model: {model_name}, CV Score: {cv_score:.4f}")

        if cv_score > best_score:
            best_score = cv_score
            best_model = model
            best_name = model_name

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model, best_name


from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

def feature_selection_with_rfe_cv(X_train, y_train, n_features, best_model):
    """
    Perform feature selection using RFE, with fallback to univariate selection for models without coefficients.
    """
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        # Use RFE for models with coefficients or feature importances
        rfe = RFE(estimator=best_model, n_features_to_select=n_features, step=1)
        rfe.fit(X_train, y_train)

        selected_features = rfe.support_
        if np.sum(selected_features) == 0:
            print("No features selected using RFE. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    else:
        # Fallback to univariate feature selection
        print("Model lacks coefficients/feature importance; using univariate feature selection.")
        
        # Use SelectKBest with F-statistic (or mutual information if preferred)
        selector = SelectKBest(score_func=f_classif, k=n_features)
        selector.fit(X_train, y_train)

        selected_features = selector.get_support()
        if np.sum(selected_features) == 0:
            print("No features selected using univariate method. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    return selected_features


# Two-step grid search for hyperparameter optimization
def tune_model_hyperparameters(model, model_name, X_train, y_train):
    refined_grid = {}  # Initialize with a default value to avoid "unbound variable" error

    if model_name == "Logistic Regression":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Ridge Classifier":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Lasso (L1)" or model_name == "ElasticNet (L1+L2)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "SVM (Linear)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Random Forest":
        broad_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
    elif model_name == "Perceptron":
        broad_param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
    elif model_name == "LDA":
        broad_param_grid = {'shrinkage': [None, 'auto'], 'solver': ['svd', 'lsqr', 'eigen']}
    else:
        raise ValueError(f"Model {model_name} does not have a defined parameter grid.")

    # Broad Grid Search
    broad_search = GridSearchCV(model, broad_param_grid, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
    broad_search.fit(X_train, y_train)
    best_params_broad = broad_search.best_params_

    # Define refined grid based on broad search results
    if model_name in ["Logistic Regression", "Lasso (L1)", "ElasticNet (L1+L2)", "SVM (Linear)"]:
        refined_grid = {'C': np.linspace(best_params_broad['C'] * 0.1, best_params_broad['C'] * 10, 5)}
    elif model_name == "Random Forest":
        refined_grid = {
            'n_estimators': [max(10, best_params_broad['n_estimators'] - 50), best_params_broad['n_estimators'], best_params_broad['n_estimators'] + 50],
            'max_depth': [None] if not best_params_broad['max_depth'] else [
                max(1, best_params_broad['max_depth'] - 5), best_params_broad['max_depth'], best_params_broad['max_depth'] + 5]
        }
    elif model_name == "Perceptron":
        refined_grid = {'alpha': np.linspace(best_params_broad['alpha'] * 0.1, best_params_broad['alpha'] * 10, 5)}
    elif model_name == "LDA":
        refined_grid = {'shrinkage': [best_params_broad['shrinkage']], 'solver': [best_params_broad['solver']]}

    # Narrow Grid Search
    narrow_search = GridSearchCV(model, refined_grid, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
    narrow_search.fit(X_train, y_train)

    print(f"Best Parameters (Broad Search): {best_params_broad}")
    print(f"Best Parameters (Narrow Search): {narrow_search.best_params_}")

    best_model = narrow_search.best_estimator_
    return best_model


# Train and evaluate final model
def train_and_evaluate_final_model(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    print(f"Test Set Accuracy (Final Model): {test_accuracy:.4f}")
    return model, test_accuracy


# Cosine similarity between two models
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_model_similarity(model1, model2):
    """
    Calculate the similarity between two models using cosine similarity or feature importances.

    Parameters:
    - model1: First trained model
    - model2: Second trained model

    Returns:
    - similarity: Cosine similarity score between the two models' coefficients or importances.
    """
    # Extract the weights (coefficients) or feature importances
    def get_model_vector(model):
        if hasattr(model, 'coef_'):  # Linear models with coefficients
            return model.coef_.flatten()
        elif hasattr(model, 'feature_importances_'):  # Tree-based models
            return model.feature_importances_
        else:
            raise ValueError(f"Model of type {type(model)} does not have coefficients or feature importances.")
    
    try:
        # Get vectors for the two models
        vector1 = get_model_vector(model1)
        vector2 = get_model_vector(model2)

        # Ensure vectors are of the same length
        if len(vector1) != len(vector2):
            raise ValueError("Model vectors have different lengths. Ensure the models were trained on the same features.")

        # Calculate cosine similarity
        similarity = cosine_similarity([vector1], [vector2])
        return similarity[0][0]  # Return the scalar similarity value

    except ValueError as e:
        print(f"Error in calculating similarity: {e}")
        return None


# Full pipeline
def pipeline(X, y, n_features):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y)

    best_model, best_name = model_selection(X_train, y_train)

    selected_features = feature_selection_with_rfe_cv(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy


# Compare CAD and PAD models
def compare_models_and_analyze_topography(X_data_set1, y_data_set1, X_data_set2, y_data_set2, n_features):
    print("Training CAD model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline(X_data_set1, y_data_set1, n_features)

    print("\nTraining PAD model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline(X_data_set2, y_data_set2, n_features)

    print("\nEvaluating CAD model on PAD dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of CAD model on PAD data: {accuracy:.4f}")

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between CAD and PAD model weights:")
    try:
        similarity = calculate_cosine_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between CAD and PAD model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between CAD and PAD model weights: {similarity}")
    return similarity

In [411]:
# Define user inputs
user_dir = "/Users/xiaoqianxiao"
project_name = "UKB"
session_ID = 2  # Specify session

In [420]:
# Run the pipeline
data_set = "past_anxiety"  # Dataset identifier
X_pad, df_PAD = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_pad = df_PAD["hospital_not_now"]
y_pad_GAD7 = df_PAD["GAD7_score"]

data_set = "current_anxiety"  # Dataset identifier
X_cad, df_CAD = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_cad = df_CAD["hospital_current_anxiety"]
y_cad_GAD7 = df_CAD["GAD7_score"]

n_features = 20
compare_models_and_analyze_topography(X_cad, y_cad, X_pad, y_pad, n_features)

Missing files for subject 1202011, session 2.
Missing files for subject 5249934, session 2.
Missing files for subject 5828197, session 2.
Missing files for subject 1137036, session 2.
Missing files for subject 5519204, session 2.
Missing files for subject 1899042, session 2.
Missing files for subject 5273114, session 2.
Missing files for subject 4874874, session 2.
Missing files for subject 4440650, session 2.
Missing files for subject 1057410, session 2.
Missing files for subject 5616943, session 2.
Missing files for subject 2521265, session 2.
Missing files for subject 2157113, session 2.
Missing files for subject 5830707, session 2.
Missing files for subject 1099415, session 2.
Missing files for subject 2442320, session 2.
Missing files for subject 5034428, session 2.
Missing files for subject 1010408, session 2.
Missing files for subject 4555539, session 2.
Missing files for subject 2329986, session 2.
Missing files for subject 5532015, session 2.
Missing files for subject 4228115,

In [441]:
data_set = "ah"  # Dataset identifier
X_ah, df_ah = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ah = df_ah["active_history"]

data_set = "ih"  # Dataset identifier
X_ih, df_ih = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ih = df_ih["inactive_history"]

data_set = "a_noh"  # Dataset identifier
X_a_noh, df_a_noh = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_a_noh = df_a_noh["active_no_history"]


Missing files for subject 5264176, session 2.
Missing files for subject 2070414, session 2.
Missing files for subject 1119086, session 2.
Missing files for subject 4666103, session 2.
Missing files for subject 5249934, session 2.
Missing files for subject 3317035, session 2.
Missing files for subject 2149481, session 2.
Missing files for subject 5767053, session 2.
Missing files for subject 2373203, session 2.
Missing files for subject 3574504, session 2.
Missing files for subject 5745396, session 2.
Missing files for subject 1562642, session 2.
Missing files for subject 2532965, session 2.
Missing files for subject 1994466, session 2.
Missing files for subject 1439161, session 2.
Missing files for subject 4729442, session 2.
Missing files for subject 5522324, session 2.
Missing files for subject 5260648, session 2.
Missing files for subject 2652700, session 2.
Missing files for subject 3088529, session 2.
Missing files for subject 5805087, session 2.
Missing files for subject 4325311,

In [442]:
n_features = 20
compare_models_and_analyze_topography(X_ah, y_ah, X_ih, y_ih, n_features)

Training CAD model...
Model: Logistic Regression, CV Score: 0.4556
Model: Ridge Classifier, CV Score: 0.4556
Model: Lasso (L1), CV Score: 0.5111
Model: ElasticNet (L1+L2), CV Score: 0.4556
Model: LDA, CV Score: 0.4667
Model: Perceptron, CV Score: 0.4556
Model: SVM (Linear), CV Score: 0.4222
Model: Random Forest, CV Score: 0.5000
Best Model: Lasso (L1) with CV score: 0.5111
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 10}
Best Parameters (Narrow Search): {'C': np.float64(25.75)}
Test Set Accuracy (Final Model): 0.4783

Training PAD model...
Model: Logistic Regression, CV Score: 0.7773
Model: Ridge Classifier, CV Score: 0.7864
Model: Lasso (L1), CV Score: 0.7673
Model: ElasticNet (L1+L2), CV Score: 0.7764
Model: LDA, CV Score: 0.7682
Model: Perceptron, CV Score: 0.7000
Model: SVM (Linear), CV Sc

np.float64(-0.24371512573122991)

In [443]:
n_features = 20
compare_models_and_analyze_topography(X_ah, y_ah, X_a_noh, y_a_noh, n_features)

Training CAD model...
Model: Logistic Regression, CV Score: 0.4556
Model: Ridge Classifier, CV Score: 0.4556
Model: Lasso (L1), CV Score: 0.5111
Model: ElasticNet (L1+L2), CV Score: 0.4556
Model: LDA, CV Score: 0.4667
Model: Perceptron, CV Score: 0.4556
Model: SVM (Linear), CV Score: 0.4222
Model: Random Forest, CV Score: 0.5222
Best Model: Random Forest with CV score: 0.5222
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 9 candidates, totalling 90 fits
Best Parameters (Broad Search): {'max_depth': 10, 'n_estimators': 50}
Best Parameters (Narrow Search): {'max_depth': 5, 'n_estimators': 50}
Test Set Accuracy (Final Model): 0.5652

Training PAD model...


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn

In [444]:
X_ih.shape

(134, 378)

In [447]:
sum(y_ih==True)/len(y_ih)

0.22388059701492538